***Решение задач классификации и регрессии в ML***

**Вариант 10** Классификация римских амфор

**Выполнила** Острова К.В. МБИ-22-1

In [35]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
  Preparing metadata (setup.py) ... done


In [36]:
import pandas as pd
import numpy as np
import pandas_profiling
import pandas_profiling
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore")

Чтение данных из источника и создание dataframe

In [37]:
train_data = pd.read_csv('/amphoraes.csv')
train_data.head()

,Fabric,Rim diameter,Rim type,Shoulder type,Handles in profile,Handles in section,Neck type,Body type,Base type,Capacity (avg l),Type,URL,Name,Subtype,Height (cm),Width (cm)
0,agora almadrava almagro cadiz campanian dresse...,0-14 6-9 7-12 7-16 7-18 8-12 9 9-10 9-11 10-11...,beaded collar everted flanged flaring pulley\ ...,carinated none/smooth rounded,arched bowed curved ear-shaped long\ vertical ...,bifid grooved ovoid/elliptical ridged round se...,broad/wide conical cylindrical hourglass none ...,cylindrical globular ovoid piriform tapered,button flared flat knobbed long\ hollow pointe...,0-10 10-15 15-20 20-25 25-30 30-35 35-40 45-50...,Dressel Gauloise Keay,string,string,string,string,string
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class,meta origin=http://file.biolab.si/datasets/amp...,meta,meta,meta,meta
2,dressel,16-22,beaded,rounded,long vertical,ovoid/elliptical,broad/wide,cylindrical,long hollow,30-35,Dressel,Dressel/DR101.jpg,DR101,Dressel 14,90-110,27-33
3,campanian,17-18,collar,carinated,long vertical,ovoid/elliptical,cylindrical,cylindrical,spike/tapered,NaN,Dressel,Dressel/DR104.jpg,DR104,Dressel 1,97-117,28-29
4,campanian,17-18,collar,carinated,long vertical,ovoid/elliptical,cylindrical,cylindrical,spike/tapered,NaN,Dressel,Dressel/DR105.jpg,DR105,Dressel 1,97-117,28-29


Определяем какие параметры имеют пропущенные значения

In [38]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Fabric              157 non-null    object
 1   Rim diameter        154 non-null    object
 2   Rim type            165 non-null    object
 3   Shoulder type       165 non-null    object
 4   Handles in profile  165 non-null    object
 5   Handles in section  165 non-null    object
 6   Neck type           164 non-null    object
 7   Body type           165 non-null    object
 8   Base type           163 non-null    object
 9   Capacity (avg l)    72 non-null     object
 10  Type                166 non-null    object
 11  URL                 166 non-null    object
 12  Name                166 non-null    object
 13  Subtype             166 non-null    object
 14  Height (cm)         143 non-null    object
 15  Width (cm)          131 non-null    object
dtypes: object(16)
memory usage

Удаляем значения NaN

In [39]:
data = train_data.drop([0])
data.isnull().sum()

Fabric                 9
Rim diameter          12
Rim type               1
Shoulder type          1
Handles in profile     1
Handles in section     1
Neck type              2
Body type              1
Base type              3
Capacity (avg l)      94
Type                   0
URL                    0
Name                   0
Subtype                0
Height (cm)           23
Width (cm)            35
dtype: int64

Обработка пропусков. Если пропусков у признака слишком много (более 70%), то как правило, такой признак удаляют. Проверим насколько полных признаков в данном наборе. Используем метод isnull(), а также метод mean() суммирует все значения True и определяем концентрацию пропусков в каждом столбце. Умножаем на 100, чтобы получить значение в процентах.

0 - пустых значение нет, если значение больше 0 - то есть пропуски.


In [40]:
data.isnull().mean()*100

Fabric                 5.454545
Rim diameter           7.272727
Rim type               0.606061
Shoulder type          0.606061
Handles in profile     0.606061
Handles in section     0.606061
Neck type              1.212121
Body type              0.606061
Base type              1.818182
Capacity (avg l)      56.969697
Type                   0.000000
URL                    0.000000
Name                   0.000000
Subtype                0.000000
Height (cm)           13.939394
Width (cm)            21.212121
dtype: float64

Обрабатываем пропуски с помощью sklearn.impute.SimpleImputer. Мы выбираем все категориальные переменные и применяем стратегию "вставить вместо пропуска самое распространенное значение":

In [41]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')

In [42]:
data["Fabric"] = imputer.fit_transform(data["Fabric"].values.reshape(-1,1))[:,0]

data["Rim diameter"] = imputer.fit_transform(data["Rim diameter"].values.reshape(-1,1))[:,0]

data["Rim type"] = imputer.fit_transform(data["Rim type"].values.reshape(-1,1))[:,0]

data["Shoulder type"] = imputer.fit_transform(data["Shoulder type"].values.reshape(-1,1))[:,0]

data["Handles in profile"] = imputer.fit_transform(data["Handles in profile"].values.reshape(-1,1))[:,0]

data["Handles in section"] = imputer.fit_transform(data["Handles in section"].values.reshape(-1,1))[:,0]

data["Neck type"] = imputer.fit_transform(data["Neck type"].values.reshape(-1,1))[:,0]

data["Body type"] = imputer.fit_transform(data["Body type"].values.reshape(-1,1))[:,0]

data["Base type"] = imputer.fit_transform(data["Base type"].values.reshape(-1,1))[:,0]

data["Capacity (avg l)"] = imputer.fit_transform(data["Capacity (avg l)"].values.reshape(-1,1))[:,0]

data["Height (cm)"] = imputer.fit_transform(data["Height (cm)"].values.reshape(-1,1))[:,0]

data["Width (cm)"] = imputer.fit_transform(data["Width (cm)"].values.reshape(-1,1))[:,0]

In [43]:
data.head(100)

,Fabric,Rim diameter,Rim type,Shoulder type,Handles in profile,Handles in section,Neck type,Body type,Base type,Capacity (avg l),Type,URL,Name,Subtype,Height (cm),Width (cm)
1,gauloise,13-14,everted,rounded,curved,ovoid/elliptical,cylindrical,cylindrical,spike/tapered,25-30,class,meta origin=http://file.biolab.si/datasets/amp...,meta,meta,meta,meta
2,dressel,16-22,beaded,rounded,long vertical,ovoid/elliptical,broad/wide,cylindrical,long hollow,30-35,Dressel,Dressel/DR101.jpg,DR101,Dressel 14,90-110,27-33
3,campanian,17-18,collar,carinated,long vertical,ovoid/elliptical,cylindrical,cylindrical,spike/tapered,25-30,Dressel,Dressel/DR104.jpg,DR104,Dressel 1,97-117,28-29
4,campanian,17-18,collar,carinated,long vertical,ovoid/elliptical,cylindrical,cylindrical,spike/tapered,25-30,Dressel,Dressel/DR105.jpg,DR105,Dressel 1,97-117,28-29
5,campanian,17-18,collar,carinated,long vertical,ovoid/elliptical,cylindrical,cylindrical,spike/tapered,25-30,Dressel,Dressel/DR106.jpg,DR106,Dressel 1,97-117,28-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,gauloise,7-18,everted,none/smooth,ring,grooved,short/narrow,globular,ringed,35-40,Gauloise,Gauloise/DR374.jpg,DR374,Gauloise 12,24-50,25-30
97,gauloise,130,flaring,none/smooth,long vertical,grooved,hourglass,ovoid,ringed,25-30,Gauloise,Gauloise/DR386.jpg,DR386,Gauloise 1 similis in Verulamium White Ware,530,380
98,almadrava,12-13,rounded,rounded,arched,grooved,short/narrow,tapered,flat,25-30,Gauloise,Gauloise/DR463.jpg,DR463,Gauloise 4 similis - Almadrava 3,60-61,41-42
99,gauloise,14-16,collar,none/smooth,curved,grooved,short/narrow,globular,ringed,25-30,Gauloise,Gauloise/DR536.jpg,DR536,Gauloise 1,47-53,38-44


# Проводим проверку распределения данных с использованием pandas profiling.



In [44]:
df = data
pandas_profiling.ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Profiling выдаёт в своём отчёте следующие параметры датасета:

*   тип данных в каждой колонке;
*   пропущенные и уникальные значения (количество и процент);
*   описательную статистику: квартили, медиану, межквартильный размах, среднее, моду, абсолютное и относительное стандартное отклонение, медианное абсолютное отклонение, коэффициенты асимметрии и эксцесса;
*   график в виде гистограммы;
*   корреляции между значениями (Пирсона, Спирмена и Кендалла);
*   матрицу пропущенных значений;
*   анализ текста в категориальных значениях;
*   а также метаданные файлов и изображений: размеры файлов, даты создания, высоту и ширину.


# Исследовательский анализ данных (EDA)

Рассмотрим отчет по корреляционному анализу. Для начала рассмотрим тесноту связи между данными. С этим поможет корреляционная матрица из предыдущего исследования. Коэффициент корреляции изменяется от -1 до +1.

Шкала Чеддока для величин коэффициента корреляции трактует связь между
наблюдаемыми величинами следующим образом:

0 – полное отсутствие связи;

0 – 0.3 – очень слабая;

3 – 0.5 – слабая;

5 – 0.7 – средняя;

7 – 0.9 – высокая;

9 – 1 – очень высокая.

1 – абсолютная взаимозависимость.

Исходя из полученной матрицы сформируем выводы о связи параеметров (для
формирования выводoв о связи между паремтров будем брать значения > |0.5|, то есть начиная от средней и заканчивая абсолютной взаимозивисимостью):

1.   Base type	положительно коррелирует с Handles in section и Type
2.   Body type положительно коррелирует с Shoulder type
1.   Capacity (avg l) положительно коррелирует с Neck type,	Rim diameter,		Shoulder type
2.   Fabric положительно коррелирует с Handles in section,	Rim diameter и Type
1.   Handles in profile положительно коррелирует с Rim diameter, Shoulder type
1.   Handles in section положительно коррелирует с Base type, Fabric, Rim diameter
2.   Neck type положительно коррелирует с Capacity (avg l), Rim diameter
2.   Rim diameter положительно коррелирует с Base type,	Body type,	Capacity (avg l),	Fabric,	Handles in profile,	Handles in section,	Neck type,	Rim type,	Shoulder type
1.   Rim type положительно коррелирует с Rim diameter
2.   Shoulder type положительно коррелирует с Body type,	Capacity (avg l),	Handles in profile, Rim diameter
1.   Type положительно коррелирует с Rim diameter











# Разделение набора данных на обучающую и тестовую выборку, выдвижение гипотезы о возможных применимых алгоритмах машинного обучения, обучение выбранных моделей, выполнение классификации или прогноза с использованием выбранной модели и тестовой выборки

Возможные алгоритмы для машинного обучения:

•Логистическая регрессия

•Дерево решений

•К ближайших соседей

•Байевский классификатор

•SVM


Метод К ближайших соседей (K-Nearest Neighbors, KNN) является простым и интуитивно понятным алгоритмом машинного обучения, который может быть применен для различных задач, таких как классификация и регрессия.

Вот несколько причин, по которым стоит выбрать метод K ближайших соседей:

1. Простота реализации: KNN - один из самых простых методов машинного обучения для понимания и реализации. Отсутствует необходимость в сложном математическом понимании или алгоритмах оптимизации.

2. Нет обучения: KNN является ленивым алгоритмом, что означает, что он не требует процесса обучения, и модель хранит только обучающий набор данных. Это позволяет легко адаптировать алгоритм для новых данных или изменений в существующих данных без необходимости переобучения.

3. Гибкость: KNN может быть использован как для задач классификации, так и для задач регрессии. Он может обрабатывать как числовые, так и категориальные признаки, а также может работать с большим количеством признаков.

4. Устойчивость к выбросам: KNN устойчив к выбросам в данных. Поскольку классификация определяется на основе ближайших соседей, выбросы не будут сильно влиять на результаты классификации или регрессии.

Когда применять метод K ближайших соседей:

1. Маленький объем данных: KNN хорошо работает с небольшими выборками данных, поскольку расчет стоимости классификации основан на расстояниях между ближайшими соседями.

2. Некоторая явная структура: KNN показывает лучшие результаты в тех случаях, когда в данных присутствует явная структура, например, когда объекты из одного класса склонны собираться вместе.

3. Отсутствие явных закономерностей: KNN может быть полезен, если данных недостаточно для создания сложной модели или если явные закономерности или зависимости между признаками отсутствуют.

Воспользуемся методом К ближайших соседей, т.к. наша задача классификации римских


амфор по ряду характеристик. Прежде чем приступить к этому, разделим датасет на два набора данных: тренировочный, который будем использовать для обучения алгоритма, и тестовый — для проверки точности его работы. Для этого воспользуемся стандартным методом train_test_split.


Для начала удалим столбцы, которые не будут участвовать в исследовании

In [45]:
data.drop(columns=['Rim diameter', 'Handles in section', 'Capacity (avg l)', 'URL', 'Height (cm)', 'Width (cm)', 'Name', 'Subtype'], inplace=True)

# Сохранение изменений в новом файле
data.to_csv('amphoraes_modified.csv', index=False)

In [46]:
new_data = pd.read_csv('/content/amphoraes_modified.csv')
new_data.head(1000)

,Fabric,Rim type,Shoulder type,Handles in profile,Neck type,Body type,Base type,Type
0,gauloise,everted,rounded,curved,cylindrical,cylindrical,spike/tapered,class
1,dressel,beaded,rounded,long vertical,broad/wide,cylindrical,long hollow,Dressel
2,campanian,collar,carinated,long vertical,cylindrical,cylindrical,spike/tapered,Dressel
3,campanian,collar,carinated,long vertical,cylindrical,cylindrical,spike/tapered,Dressel
4,campanian,collar,carinated,long vertical,cylindrical,cylindrical,spike/tapered,Dressel
...,...,...,...,...,...,...,...,...
160,tunisian,rounded,rounded,ear-shaped,conical,cylindrical,knobbed,Keay
161,tunisian,rounded,rounded,ring,cylindrical,piriform,button,Keay
162,tunisian,everted,rounded,curved,hourglass,cylindrical,button,Keay
163,tunisian,everted,rounded,curved,hourglass,cylindrical,button,Keay


Каждому значению в таблице присваиваем числовое значение, которое будет  идентифицировать

In [47]:
new_data = pd.read_csv('/content/amphoraes_modified.csv')
# Словарь с соответствиями числовых значений и их замен
value_mapping = {
    'gauloise': 1,
    'dressel': 2,
    'campanian': 3,
    'keay': 4,
    'pascual':5,
    'almadrava':6,
    'lake mariout':7,
    'lyon':8,
    'cadiz':9,
    'tunisian':10,
    'everted':11,
    'beaded':12,
    'collar':13,
    'rounded':14,
    'triangular':15,
    'pulley wheel':16,
    'flaring':17,
    'flanged':18,
    'carinated':19,
    'none/smooth':20,
    'class':21,
    'curved':22,
    'long vertical':23,
    'ring':24,
    'short vertical':25,
    'peaked':26,
    'arched':27,
    'ear-shaped':28,
    'bowed':29,
    'cylindrical':30,
    'broad/wide':31,
    'hourglass':32,
    'short/narrow':33,
    'conical':34,
    'ovoid':35,
    'globular':36,
    'tapered':37,
    'piriform':38,
    'spike/tapered':39,
    'long hollow':40,
    'flat':41,
    'knobbed':42,
    'short hollow':43,
    'pointed':44,
    'button':45,
    'flared':46,
    '24ed':47,

}

# Замена значений в таблице
data = data.replace(value_mapping)
# Сохранение изменений в новом файле
data.to_csv('amphoraes_modified_neww.csv', index=False)


In [55]:

import pandas as pd # data processing
from termcolor import colored as cl # elegant printing of text
import seaborn as sb # visualizations
import matplotlib.pyplot as plt # editing visualizations
from matplotlib import style # setting styles for plots
from sklearn.preprocessing import StandardScaler # normalizing data
from sklearn.neighbors import KNeighborsClassifier # KNN algorithm
from sklearn.metrics import accuracy_score # algorithm accuracy
from sklearn.model_selection import train_test_split # splitting the data

style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (16, 7)

In [56]:
new_data1 = pd.read_csv('/content/amphoraes_modified_neww.csv')
new_data1.head(1000)
print(cl(new_data1, attrs = ['bold']))

     Fabric  Rim type  Shoulder type  Handles in profile  Neck type  \
0         1        11             14                  22         30   
1         2        12             14                  23         31   
2         3        13             19                  23         30   
3         3        13             19                  23         30   
4         3        13             19                  23         30   
..      ...       ...            ...                 ...        ...   
160      10        14             14                  28         34   
161      10        14             14                  24         30   
162      10        11             14                  22         32   
163      10        11             14                  22         32   
164      10        11             14                  22         32   

     Body type  Base type  Type  
0           30         39    21  
1           30         40     2  
2           30         39     2  
3          

Определяем переменную «X» и переменную «Y», чтобы построить нашу модель KNN. Учитывая наш набор данных, нам нужно классифицировать переменную, поэтому ее можно использовать как переменную «Y» или как зависимую переменную. Все остальные переменные в нашем наборе данных можно рассматривать как независимые переменные или переменные «X».

In [58]:
# Разделение данных на признаки (X) и целевую переменную (y)
X_var = new_data1[['Fabric', 'Rim type', 'Shoulder type', 'Handles in profile','Neck type','Body type','Base type' ]].values
y_var = new_data1['Type'].values

print(cl('X variable :', attrs = ['bold']), X_var[:5])
print(cl('Y variable :', attrs = ['bold']), y_var[:5])


X variable : [[ 1 11 14 22 30 30 39]
 [ 2 12 14 23 31 30 40]
 [ 3 13 19 23 30 30 39]
 [ 3 13 19 23 30 30 39]
 [ 3 13 19 23 30 30 39]]
Y variable : [21  2  2  2  2]


Теперь нам нужно нормализовать наши значения переменной «X», которые могут быть полезны при обучении нашей модели KNN. До этого нормализация - это процесс структурирования и удаления аномалий для заданного набора значений. Это также снижает количество данных и улучшает целостность данных. Чтобы нормализовать значения, мы можем использовать функцию ‘StandardScaler’ в scikit-learn.

In [59]:
X_var = StandardScaler().fit(X_var).transform(X_var.astype(float))
print(cl(X_var[:5], attrs = ['bold']))

[[-0.95852347 -0.97555019 -0.74127426 -1.01897102 -1.0606781  -1.06788686
  -0.11089842]
 [-0.69000671 -0.48330009 -0.74127426 -0.57300227 -0.38233745 -1.06788686
   0.02266537]
 [-0.42148995  0.00895     1.15207331 -0.57300227 -1.0606781  -1.06788686
  -0.11089842]
 [-0.42148995  0.00895     1.15207331 -0.57300227 -1.0606781  -1.06788686
  -0.11089842]
 [-0.42148995  0.00895     1.15207331 -0.57300227 -1.0606781  -1.06788686
  -0.11089842]]


Теперь, когда у нас есть идеальные зависимые и независимые переменные. Теперь мы можем продолжить обучение нашей модели KNN. Чтобы обучить нашу модель, мы должны сначала разделить наши данные на обучающий набор и набор для тестирования, где обучающий набор имеет наибольшее количество точек данных. Чтобы разделить наши данные, мы можем использовать функцию ‘train_test_split’, предоставляемую scikit-learn на python.

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_var, y_var, test_size = 0.3, random_state = 0)

print(cl('Train set shape :', attrs = ['bold']), X_train.shape, y_train.shape)
print(cl('Test set shape :', attrs = ['bold']), X_test.shape, y_test.shape)

Train set shape : (115, 7) (115,)
Test set shape : (50, 7) (50,)


В приведенном выше коде мы использовали train_test_split, чтобы разделить наши данные на обучающий набор и набор для тестирования. Внутри функции мы указали, что наш тестовый набор должен состоять из 30% данных, а остальное - это обучающий набор. Наконец, мы упомянули, что не должно быть случайного перетасовки наших данных при разделении.

In [61]:
k = 3

neigh = KNeighborsClassifier(n_neighbors = k)
neigh.fit(X_train, y_train)

print(cl(neigh, attrs = ['bold']))

KNeighborsClassifier(n_neighbors=3)


Во-первых, мы указали для нашего значения «K» значение 3. Затем мы определили наш алгоритм и, наконец, подогнали наши значения набора параметров к алгоритму. После распечатки алгоритма мы видим, что «metric = minkowski» - это не что иное, как указание на то, что метод, используемый для вычисления расстояния до соседей, - это метод Минковского. Существуют также другие методы, такие как метод евклидова расстояния и метод индекса Жаккара, но его нужно определять вручную.

После завершения обучения нашего алгоритма KNN, предскажем тестовые значения с помощью нашего обученного алгоритма и оценим наши результаты, используя оценочные показатели scikit-learn.

In [62]:
yhat = neigh.predict(X_test)

print(cl('Prediction Accuracy Score (%) :', attrs = ['bold']), round(accuracy_score(y_test, yhat)*100, 2))

Prediction Accuracy Score (%) : 86.0


 Акктуальное и Предсказанное значение:

In [67]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': yhat})
print(results)

    Actual  Predicted
0        2          2
1        2          1
2        1          4
3        2          2
4        2          2
5        2          2
6        2          2
7        2          2
8        4          4
9        4          4
10       4          4
11       1          1
12       2          2
13       4          4
14       4          4
15       4          4
16       1          1
17       4          4
18       1          1
19       4          2
20       4          4
21       2          2
22       1          1
23       4          4
24       2          2
25       2          2
26       2          2
27       2          2
28       1          1
29       2          2
30       2          2
31       2          2
32       2          2
33       2          2
34       2          2
35       4          4
36       1          4
37       2          2
38       1          1
39       4          4
40       2          2
41       2          4
42       1          1
43       2          2
44       4

Отличие Актуального и Предсказанного значения не отличается более чем на 3



Используя наш обученный алгоритм KNN, мы предсказали значения тестового набора. Использовали метрику оценки «precision_score», чтобы проверить точность наших прогнозируемых результатов. В выходных данных мы видим, что результаты имеют точность 86%, что означает, что наша модель KNN очень хорошо работает и для данного набора, она способна решать реальные проблемы классификации. Таким образом, мы успешно построили, обучили и оценили нашу модель KNN на Python.

# Оценить точность модели с использованием таких метрик как MAE , MSE , построить confusion matrix

Оценим полученные результаты с помощью средней квадратической ошибки (MSE) и средней абсолютной ошибки (MAE)

Оценим полученные результаты двумя метриками: средней квадратической ошибкой (MSE) и средней абсолютной ошибкой (MAE).

MSE (Mean Squared Error) представляет собой среднее значение квадратов ошибок, то есть разницы между предсказанием модели и фактическим значением. Эта метрика полезна для выявления аномалий. **Чем ближе значение MSE к нулю, тем более точной считается модель.** Однако "хорошего" значения MSE нет, так как оно зависит от конкретного набора данных и может использоваться только для сравнения с предыдущими запусками модели.

MAE (Mean Absolute Error) показывает, насколько близки предсказания модели к фактическим значениям. Эта метрика менее чувствительна к выбросам и дает общую оценку качества модели. **Чем ближе значение MAE к нулю, тем более точной считается модель.** MAE измеряет среднюю абсолютную ошибку между прогнозами модели и целевыми значениями, поэтому её значение нужно минимизировать. Значение MAE зависит от шкалы целевой переменной, поэтому нет определенного значения, которое можно считать высоким или низким, и его следует интерпретировать в контексте конкретной задачи.

Строки в матрице путаницы представляют фактические значения, а столбцы - прогнозируемые значения. Диагональ сверху вниз показывает правильно классифицированные выборки.

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix

In [66]:
# Вычисление MAE и MSE
mae = mean_absolute_error(y_test, yhat)
mse = mean_squared_error(y_test, yhat)

# Построение confusion matrix
cm = confusion_matrix(y_test, yhat)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Confusion Matrix:")
print(cm)

Mean Absolute Error (MAE): 0.28
Mean Squared Error (MSE): 0.64
Confusion Matrix:
[[ 9  1  2]
 [ 1 22  1]
 [ 0  2 12]]


В результате, можно сказать, что модель счиатется более точной, т.к значения близки к 0 и равны (MAE): 0.28, (MSE): 0.64